In [79]:
#Consensus splice sites
#Donor ('[AC]AGGU[AG]AGU')
#('[GU]AGGU[AG]AGU')
#('[AC][CGU]GGU[AG]AGU')
#('[AC]A[ACU]GU[AG]AGU')
#('[AC]AG[ACU]U[AG]AGU')
#('[AC]AGG[ACG][AG]AGU')
#('[AC]AGGU[CU]AGU')
#('[AC]AGGU[AG][CGU]GU')
#('[AC]AGGU[AG]A[ACU]U')
#('[AC]AGGU[AG]AG[ACG]')


#branch point, spacer, acceptor [CU]U[AG]AC.{20,50}CAGG 
#('[AG]U[AG]AC.{20,50}CAGG')
#('[CU][ACG][AG]AC.{20,50}CAGG')
#('[CU]U[CU]AC.{20,50}CAGG')
#('[CU]U[AG][CGU]C.{20,50}CAGG')
#('[CU]U[AG]A[AGU].{20,50}CAGG')
#('[CU]U[AG]AC.{20,50}[AGU]AGG')
#('[CU]U[AG]AC.{20,50}C[CGU]GG')
#('[CU]U[AG]AC.{20,50}CA[ACU]G')
#('[CU]U[AG]AC.{20,50}CAG[ACU]')
#This set searches for seqs that deviate by one nt


#file = open(a DNA sequence)
#to not perturb exon2 acceptor splice site do not codon optimise the first 10 aa residues of it. 
#do still mutate each of those amino acids to every other

#search the dna sequence to find all the occurences of donor splice site

#for each occurence return the codons that make it
#sequentially mutate the codons and assess whether the splice site remains the real donor site differs by 
#two residues from consensus...should this be the cut off?

import re

codondicts=[{'A': '4', 'C': '2', 'D': '2', 'E': '2', 'F': '2', 'G': '4', 'H': '2', 'I': '3', 'K': '2', 'L': '6', 'M': '1', 'N': '2', 'P': '4', 'Q': '2', 'R': '6', 'S': '6', 'T': '4', 'V': '4', 'W': '1', 'Y': '2'}
,{'A': 'GCC', 'C': 'UGU', 'D': 'GAU', 'E': 'GAG', 'F': 'UUU', 'G': 'GGC', 'H': 'CAC', 'I': 'AUC', 'K': 'AAG', 'L': 'CUG', 'M': 'AUG', 'N': 'AAC', 'P': 'CCU', 'Q': 'CAG', 'R': 'AGA', 'S': 'UCU', 'T': 'ACC', 'V': 'GUG', 'W': 'UGG', 'Y': 'UAC'}
,{'A': 'GCU', 'C': 'UGC', 'D': 'GAC', 'E': 'GAA', 'F': 'UUC', 'G': 'GGA', 'H': 'CAU', 'I': 'AUU', 'K': 'AAA', 'L': 'CUC', 'N': 'AAU', 'P': 'CCC', 'Q': 'CAA', 'R': 'AGG', 'S': 'UCC', 'T': 'ACA', 'V': 'GUC', 'Y': 'UAU'}
,{'A': 'GCA', 'G': 'GGG', 'I': 'AUA', 'L': 'UUG', 'P': 'CCA', 'R': 'CGG', 'S': 'AGC', 'T': 'ACU', 'V': 'GUU'}
,{'A': 'GCG', 'G': 'GGU', 'L': 'CUU', 'P': 'CCG', 'R': 'CGC', 'S': 'AGU', 'T': 'ACG', 'V': 'GUA'}
,{'L': 'CUA', 'R': 'CGA', 'S': 'UCA'}
,{'L': 'UUA', 'R': 'CGU', 'S': 'UCG'}
,{'UUU': 'F', 'UUC': 'F', 'UUA': 'L', 'UUG': 'L', 'UCU': 'S', 'UCC': 'S', 'UCA': 'S', 'UCG': 'S', 'UAU': 'Y', 'UAC': 'Y', 'UAA': '*', 'UAG': '*', 'UGU': 'C', 'UGC': 'C', 'UGA': '*', 'UGG': 'W', 'CUU': 'L', 'CUC': 'L', 'CUA': 'L', 'CUG': 'L', 'CCU': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P', 'CAU': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q', 'CGU': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R', 'AUU': 'I', 'AUC': 'I', 'AUA': 'I', 'AUG': 'M', 'ACU': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T', 'AAU': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K', 'AGU': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R', 'GUU': 'V', 'GUC': 'V', 'GUA': 'V', 'GUG': 'V', 'GCU': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A', 'GAU': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E', 'GGU': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'}]

consensus_d_and_mt1=[('[AC]AGGU[AG]AGU'),('[GU]AGGU[AG]AGU'),('[AC][CGU]GGU[AG]AGU'),('[AC]A[ACU]GU[AG]AGU'),('[AC]AG[ACU]U[AG]AGU'),('[AC]AGG[ACG][AG]AGU'),('[AC]AGGU[CU]AGU'),('[AC]AGGU[AG][CGU]GU'),('[AC]AGGU[AG]A[ACU]U'),('[AC]AGGU[AG]AG[ACG]')]
consensus_bsa_and_mt1=[('[CU]U[AG]AC.{20,50}CAGG'),('[AG]U[AG]AC.{20,50}CAGG'),('[CU][ACG][AG]AC.{20,50}CAGG'),('[CU]U[CU]AC.{20,50}CAGG'),('[CU]U[AG][CGU]C.{20,50}CAGG'),('[CU]U[AG]A[AGU].{20,50}CAGG'),('[CU]U[AG]AC.{20,50}[AGU]AGG'),('[CU]U[AG]AC.{20,50}C[CGU]GG'),('[CU]U[AG]AC.{20,50}CA[ACU]G'),('[CU]U[AG]AC.{20,50}CAG[ACU]')]

file = open('/Users/na395/Desktop/Python files/prot_seq_and_no_run_dna.txt','r')

dnaseqcont = file.readline().rstrip('\n')

file.close()

file = open('/Users/na395/Desktop/Python files/prot_seq_and_no_run_no_cryptic_dna.txt','w')

dnaseq = [('NNN')]
protseq = ('')
for x in range(int(len(dnaseqcont)/3)):
    codon = dnaseqcont[3*x:3*x+3]
    dnaseq.append(codon)
    aa = codondicts[7][codon]
    protseq = protseq+aa
dnaseq.append('NNN')
dnaseqcont = ('NNN') + dnaseqcont + ('NNN')

matches = []
#return all the matches for consensus_mt1 in dnaseqcont



def count_cryptic_d(seq):
    cryptic_d_numbers = 0
    for each in consensus_d_and_mt1:
        cryptic_d_numbers = cryptic_d_numbers + len(re.findall(each, seq))   
    return(cryptic_d_numbers)

def count_cryptic_bsa(seq):
    cryptic_bsa_numbers = 0
    for each in consensus_bsa_and_mt1:
        cryptic_bsa_numbers = cryptic_bsa_numbers + len(re.findall(each, seq))   
    return(cryptic_bsa_numbers)

print(count_cryptic_d(dnaseqcont))
print(count_cryptic_bsa(dnaseqcont))

#this next bit makes list of match objects for cryptic elements found
#had to use the temp and double for loop to extract double match objects...

temp = []
for each in consensus_d_and_mt1:
    temp.append(re.finditer(each,dnaseqcont))

cryptic_d_matches = []    
for each in temp:
    for every in each:
        cryptic_d_matches.append(every)    
    
temp = []
for each in consensus_bsa_and_mt1:
    temp.append(re.finditer(each,dnaseqcont))

cryptic_bsa_matches = []    
for each in temp:
    for every in each:
        cryptic_bsa_matches.append(every)

    
for all in cryptic_d_matches:
    
    firstcodon=((all.start()-all.start()%3)/3)
    lastcodon=(((all.end()-1)-(all.end()-1)%3)/3) #need to +1 if you want this to be part of a range

    tempcodons=[] #need tempcodons because it can be used to permanently change a codon and then trial changes of the others
    tempcodons=dnaseq[int(firstcodon)-1:int(lastcodon)+2]
    temptempcodons=tempcodons   #need temptempcodons because it can be used to try single changes of codons

    for z in '123456': #23456 because that is how many codon dictionaries there are
            if count_cryptic_d(''.join(temptempcodons)) == 0: #this is to stop the perm loop once the temp loop 
                break
            else:
                w=(0) #for marking which codon we are at
                tempcodons=dnaseq[int(firstcodon)-1:int(lastcodon)+2]
                for every in protseq[int(firstcodon)-1:int(lastcodon)]:
                    w = int(w)+1
                    if int(z)-1 >= int(codondicts[0][every]):
                        #it means that all the permanent codons for this amino acid have been tried 
                        continue
                    else:
                        #set that codon to perm and then assess
                        newpermcodon= (codondicts[int(z)][every])
                        tempcodons[w]=newpermcodon
                        for x in '123456': #23456 because that is how many codon dictionaries there are
                                if count_cryptic_d(''.join(temptempcodons)) == 0:
                                    break
                                y=(0)         #for marking which codon we are at

                                for each in protseq[int(firstcodon)-1:int(lastcodon)]:
                                    y=int(y)+1
                                    if int(x)-1 >= int(codondicts[0][each]):                            
                                        #it means that all the temporary codons for this amino acid have been tried 
                                        continue
                                    else:
                                        temptempcodons=tempcodons #this resets temptempcodons to the original
                                        newcodon = (codondicts[int(x)][each])
                                        temptempcodons[y]=newcodon
                                        if count_cryptic_d(''.join(temptempcodons)) == 0:
                                            break

                                        else:                             

                                            pass
                                            #it  means that the cryptic donor has NOT been removed
                                            #it means that further codon mts need to be attempted


        #hopefully temptempcodons can now be used to replace the original dnaseq



    if count_cryptic_d(''.join(temptempcodons)) == 0:
            #take these mutations and enter into original sequence
            for x in range(len(temptempcodons)-2):
                dnaseq[int(firstcodon)+int(x)]=temptempcodons[int(x)+1]



dnaseqcont=''.join(dnaseq)                
print('d ' + str(count_cryptic_d(dnaseqcont)))    
    

    
    
    
for all in cryptic_bsa_matches:
    
    firstcodon=((all.start()-all.start()%3)/3)
    lastcodon=(((all.end()-1)-(all.end()-1)%3)/3) #need to +1 if you want this to be part of a range

    tempcodons=[] #need tempcodons because it can be used to permanently change a codon and then trial changes of the others
    tempcodons=dnaseq[int(firstcodon)-1:int(lastcodon)+2]
    temptempcodons=tempcodons   #need temptempcodons because it can be used to try single changes of codons

    for z in '123456': #23456 because that is how many codon dictionaries there are
            if count_cryptic_bsa(''.join(temptempcodons)) == 0: #this is to stop the perm loop once the temp loop 
                break
            else:
                w=(0) #for marking which codon we are at
                tempcodons=dnaseq[int(firstcodon)-1:int(lastcodon)+2]
                for every in protseq[int(firstcodon)-1:int(lastcodon)]:
                    w = int(w)+1
                    if int(z)-1 >= int(codondicts[0][every]):
                        #it means that all the permanent codons for this amino acid have been tried 
                        continue
                    else:
                        #set that codon to perm and then assess
                        newpermcodon= (codondicts[int(z)][every])
                        tempcodons[w]=newpermcodon
                        for x in '123456': #23456 because that is how many codon dictionaries there are
                                if count_cryptic_bsa(''.join(temptempcodons)) == 0:
                                    break
                                y=(0)         #for marking which codon we are at

                                for each in protseq[int(firstcodon)-1:int(lastcodon)]:
                                    y=int(y)+1
                                    if int(x)-1 >= int(codondicts[0][each]):                            
                                        #it means that all the temporary codons for this amino acid have been tried 
                                        continue
                                    else:
                                        temptempcodons=tempcodons #this resets temptempcodons to the original
                                        newcodon = (codondicts[int(x)][each])
                                        temptempcodons[y]=newcodon
                                        if count_cryptic_bsa(''.join(temptempcodons)) == 0:
                                            break

                                        else:                             

                                            pass
                                            #it  means that the cryptic branch/spacer/acceptor has NOT been removed
                                            #it means that further codon mts need to be attempted


        #hopefully temptempcodons can now be used to replace the original dnaseq



    if count_cryptic_bsa(''.join(temptempcodons)) == 0:
            #take these mutations and enter into original sequence
            for x in range(len(temptempcodons)-2):
                dnaseq[int(firstcodon)+int(x)]=temptempcodons[int(x)+1]



dnaseqcont=''.join(dnaseq)                
print('bsa ' + str(count_cryptic_bsa(dnaseqcont)))


dnaseq.remove('NNN')
dnaseq.remove('NNN')               


file.write(dnaseqcont.strip('N')+'\n')


file.close()









0
13
d 0
bsa 0
